Important Libraries

In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import colorchooser
import threading
from threading import Thread
import time
import pygame  #playsound library made an error so we use this lib for handling multimedia (play sound here)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Global Variables

In [2]:
timers = []   #list to store multiple timer instances
global_sound_enabled = True #Default Timer Sound Effect

Choose Color

In [3]:
def choose_color(timer):
    color = colorchooser.askcolor()[1]
    if color:
        timer["frame"].config(bg=color)
        timer["timer_label"].config(bg=color)

Complete Notification

In [4]:
def notify_completion():
    pygame.mixer.music.load(r"D:\Computer Science\collage\Level 4\1\parallel\New folder\alarm.mp3")
    pygame.mixer.music.play()
    time.sleep(1)
    all_threads_len = threading.active_count()
    if all_threads_len > 1:
        pygame.mixer.music.load(r"D:\Computer Science\collage\Level 4\1\parallel\New folder\timer.mp3")

Play Timer Sound Effect

In [5]:
def play_timer_sound(timer):
    if timer["sound_enabled"].get():
        pygame.mixer.music.load(r"D:\Computer Science\collage\Level 4\1\parallel\New folder\timer.mp3")
        while timer["running"]:
            pygame.mixer.music.play()
            time.sleep(1)

Run Timer

In [6]:
def run_timer(timer):
    sound_thread = Thread(target=play_timer_sound, args=(timer,), daemon=True)
    sound_thread.start()
    while timer["remaining_time"] > 0 and timer["running"]:
        hours, remainder = divmod(timer["remaining_time"], 3600)
        minutes, seconds = divmod(remainder, 60)
        timer_format = f"{hours:02}:{minutes:02}:{seconds:02}"
        timer["timer_label"].config(text=timer_format)
        timer["remaining_time"] -= 1
        time.sleep(1)

    if timer["remaining_time"] == 0:
        timer["running"] = False
        timer["timer_label"].config(text="Time's Up!")
        notify_completion()

Pause & Resume Timer

In [7]:
def pause_timer(timer, pause_button):
    if timer["remaining_time"] > 0:
        timer["running"] = False
        pause_button.config(text="Resume", bg="#fcfe82", command=lambda: resume_timer(timer, pause_button))
        
def resume_timer(timer, pause_button):
    if timer["remaining_time"] > 0:
        timer["running"] = True
        pause_button.config(text="Pause", bg="yellow", command=lambda: pause_timer(timer, pause_button))
        run_thread = Thread(target=run_timer, args=(timer,), daemon=True)
        run_thread.start()

Start Timer

In [8]:
def start_timer(timer, pause_button):
    if not timer["running"]:
        if timer["remaining_time"] == 0:  
            hours = timer["enter_hours"].get()
            minutes = timer["enter_minutes"].get()
            seconds = timer["enter_seconds"].get()

            if not hours.isdigit() or not minutes.isdigit() or not seconds.isdigit():
                raise ValueError("Please enter valid numeric values.")

            hours = int(hours)
            minutes = int(minutes)
            seconds = int(seconds)

            timer["remaining_time"] = hours * 3600 + minutes * 60 + seconds

            if timer["remaining_time"] <= 0:
                raise ValueError("Time must be greater than zero.")

        timer["running"] = True
        pause_button.config(text="Pause", bg="yellow", command=lambda: pause_timer(timer, pause_button))
        run_thread = Thread(target=run_timer, args=(timer,), daemon=True)
        run_thread.start()

Reset Timer

In [9]:
def reset_timer(timer):
    timer["running"] = False
    timer["timer_label"].config(text="00:00:00")
    for entry in [timer["enter_hours"], timer["enter_minutes"], timer["enter_seconds"]]:
        entry.delete(0, tk.END)
        entry.insert(0, "0")

Delete Timer

In [10]:
def delete_timer(timer):
    timer["running"] = False 
    timer["frame"].destroy() 
    timers.remove(timer)  

Add Timer 

In [11]:
def add_timer(content_frame):
    timer_frame = tk.Frame(content_frame, bg="white", borderwidth=2, relief="groove")
    timer_frame.pack(pady=10, padx=25, fill="x", expand=True)

    timer = {
        "frame": timer_frame,
        "running": False,
        "remaining_time": 0,
        "sound_enabled": tk.BooleanVar(value=True),
    }

    tk.Label(timer_frame, text="Timer Name:", font=("Arial", 14)).grid(row=0, column=0, padx=10)
    tk.Entry(timer_frame, font=("Arial", 14), width=26).grid(row=0, column=1, padx=5)

    tk.Label(timer_frame, text="Duration (h:m:s):", font=("Arial", 14)).grid(row=0, column=2, padx=5)

    timer["enter_hours"] = tk.Entry(timer_frame, font=("Arial", 14), width=4)
    timer["enter_hours"].insert(0, "0")
    timer["enter_hours"].grid(row=0, column=3, padx=5)

    timer["enter_minutes"] = tk.Entry(timer_frame, font=("Arial", 14), width=4)
    timer["enter_minutes"].insert(0, "0")
    timer["enter_minutes"].grid(row=0, column=4, padx=5)

    timer["enter_seconds"] = tk.Entry(timer_frame, font=("Arial", 14), width=4)
    timer["enter_seconds"].insert(0, "0")
    timer["enter_seconds"].grid(row=0, column=5, padx=5)

    tk.Checkbutton(timer_frame, text="Enable Sound", bg="#d4d6fa", font=("Arial", 10), variable=timer["sound_enabled"]).grid(row=0, column=6, padx=7, pady=5)

    tk.Button(timer_frame, text="Choose Color", bg="#deccfc",font=("Arial", 10), command=lambda: choose_color(timer)).grid(row=0, column=7, padx=7)

    tk.Button(timer_frame, text="Start",font=("Arial", 10), command=lambda: start_timer(timer, pause_button), bg="lightgreen").grid(row=0, column=8, padx=7)

    pause_button = tk.Button(timer_frame, text="Pause",font=("Arial", 10), bg="yellow")
    pause_button.grid(row=0, column=9, padx=7)

    tk.Button(timer_frame, text="Reset",font=("Arial", 10), command=lambda: reset_timer(timer), bg="red").grid(row=0, column=10, padx=7)

    tk.Button(timer_frame, text="Delete",font=("Arial", 10), command=lambda: delete_timer(timer), bg="gray").grid(row=0, column=11, padx=7)


    timer["timer_label"] = tk.Label(timer_frame, text="00:00:00", font=("Arial", 18), bg="white")
    timer["timer_label"].grid(row=1, column=0, columnspan=12, pady=10)

    timers.append(timer)


Main Application

In [12]:
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Multi-Timer Application")
    root.geometry("800x600")
    pygame.mixer.init()

    
    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    content_frame = ttk.Frame(canvas)
    
    screen_width = root.winfo_screenwidth()

    canvas.create_window((0, 0), window=content_frame, anchor="nw", width=screen_width)
    
    add_timer_button = tk.Button(canvas, text="Add Timer", command=lambda: add_timer(content_frame), bg="lightblue", font=("Arial", 18))
    add_timer_button.pack(side=tk.BOTTOM , pady=20)
    
    # To resize scroll region when I add many timers
    def resize_canvas(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    content_frame.bind("<Configure>", resize_canvas)

    root.mainloop()